In [83]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

L = 20
dx = 1.0
dt = 0.01
epsilon = 1.0
time_steps = 700

np.random.seed(0)
u = np.random.uniform(-0.5, 0.5, (L, L))

x = np.linspace(0, L * dx, L)
y = np.linspace(0, L * dx, L)
X, Y = np.meshgrid(x, y)
X = X.flatten()
Y = Y.flatten()

total_x = []
total_y = []
total_u = []
total_t = []

def allen_cahn_step(u, epsilon, dx, dt):
    laplacian = (
        np.roll(u, 1, axis=0) + np.roll(u, -1, axis=0) +
        np.roll(u, 1, axis=1) + np.roll(u, -1, axis=1) -
        4 * u
    ) / dx**2
    reaction = u**3 - u
    return u + dt * (epsilon * laplacian - reaction)


frames = []
for t in range(time_steps):
    u = allen_cahn_step(u, epsilon, dx, dt)
    if t % 10 == 0:
        frames.append(u.copy())
        
        total_x.append(X)
        total_y.append(Y)
        total_u.append(u.copy().ravel())
        total_t.append(np.ones_like(X) * t)
        
total_x = np.array(total_x).flatten()
total_y = np.array(total_y).flatten()
total_t = np.array(total_t).flatten()
total_u = np.array(total_u).flatten()
total_f = total_u**3 - total_u

file_saved_data = np.concatenate((total_x.reshape(-1,1), total_y.reshape(-1,1), total_t.reshape(-1,1), total_u.reshape(-1,1), total_f.reshape(-1,1)), axis=1)
df = pd.DataFrame(file_saved_data, columns=["x", "y", "t", "u", "f"])
df.to_csv("Allen_Cahn_data.csv", index=False)

fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(frames[0], cmap='RdBu', origin='lower', animated=True)
ax.set_title("Allen-Cahn Simulation")
ax.axis('off')

def update(frame):
    im.set_array(frame)
    return [im]

ani = FuncAnimation(fig, update, frames=frames, interval=50, blit=True)
plt.close(fig)  # 애니메이션 외의 플롯 닫기


from IPython.display import HTML
HTML(ani.to_jshtml())

In [85]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

np.random.seed(0)
torch.manual_seed(0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
file = pd.read_csv("Allen_Cahn_data.csv")

model = torch.load("Allen_Cahn_model.pth", map_location='cpu')
model.eval()

frames = []

for i in range(700):
    if i % 10 == 0:
        filtered_file = file[file['t'] == i]
        x = filtered_file['x'].values
        y = filtered_file['y'].values
        t = filtered_file['t'].values
        
        inputs = np.concatenate((x.reshape(-1,1), y.reshape(-1,1), t.reshape(-1,1)), axis=1)
        inputs = torch.tensor(inputs, dtype=torch.float32)
        outputs, mu = model(inputs.to(device))
        u = outputs[:, 0].reshape(-1, 1)
        
        x_unique = np.unique(x)
        y_unique = np.unique(y)
        
        x_grid, y_grid = np.meshgrid(x_unique, y_unique)
        h_grid = u.reshape(len(x_unique), len(y_unique)).detach().numpy()
        
        frames.append(h_grid.copy())
        

fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(frames[0], cmap='RdBu', origin='lower', animated=True)
ax.set_title("Allen-Cahn Simulation")
ax.axis('off')

def update(frame):
    im.set_array(frame)
    return [im]

ani = FuncAnimation(fig, update, frames=frames, interval=50, blit=True)
plt.close(fig)  # 애니메이션 외의 플롯 닫기


from IPython.display import HTML
HTML(ani.to_jshtml())